# xCOLUMNs Quick Usage Guide

Welcome to the xCOLUMNs Quick Usage Guide. It aims to provide a quick overview of the xCOLUMNs library and how to use it.

## What is xCOLUMNs?

xCOLUMNs stands for x**Consistent Optimization of Label-wise Utilities in Multi-label classificatioN**s.
It is a small Python library that aims to implement different methods for the optimization of a general family of 
metrics that can be defined on multi-label classification matrices. 
These include but are not limited to, label-wise metrics (see below for details). 
The library provides an efficient implementation of the different optimization methods that easily scale to the extreme multi-label classification (XMLC) - problems with a very large number of labels and instances.

### What is multi-label classification?
Multi-label classification is a type of classification task, where each instance can be assigned to multiple labels. 
For example, a news article can be assigned to multiple categories, such as "politics", "sports", "technology", etc.
The labels for each instance are usually represented as binary vectors $y \in \{0, 1\}^m$, where $m$ is the number of possible labels, and $y_j = 1$ means that label $j$ is relevant for the given instance, and $y_j = 0$ otherwise.

### What are label-wise utilities?
The label-wise utilities are performance metrics that decompose over labels, i.e., they can be defined as a sum of individual metrics for each label:
$$
    \Psi(Y, \hat{Y}) = \sum_{j=1}^{m} \psi^j(y_{:,j}, \hat{y}_{:,j}) \,,
$$
where $Y$ is the true label matrix, $\hat{Y}$ is the predicted label matrix (both of size $n \times m$, where $n$ is number of instances) and $y_{:,j}$ is the $j$-th column of $Y$, and $\psi^j$ is binary utility function for label $j$.

Such metrics, under the assumption that are order-invariant, can be defined as a function of the confusion matrix, and can be decomposed over labels.

### What methods are implemented in xCOLUMNs?
The aim of xCOLUMNs is to provide methods for optimization of general family of label-wise utilities. Currently, the following methods are implemented:

- Prediction for provided test set using **Block Coordinate Ascent/Descent (BC)** method, described in [1].
- Search for optimal population classifier using **Frank-Wolfe (FW)** method, described in [2].

## How to use xCOLUMNs?
This guide will provide a quick overview of how to use xCOLUMNs for the optimization of label-wise utilities.


In [ ]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

seed=2024
x, y = make_multilabel_classification(n_samples=100000, n_features=100, n_classes=20, n_labels=3, length=50, allow_unlabeled=True, sparse=False, return_indicator='dense', return_distributions=False, random_state=seed)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=seed)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=seed)
clf = MultiOutputClassifier(LogisticRegression()).fit(x_train, y_train)

def clf_predict(clf, x):
    """
    Process the output of a multioutput classifier to get the marginal probabilities of each label (class).
    """
    y_proba = clf.predict_proba(x)
    return np.array(y_proba)[:,:,1].transpose()

y_proba_val = clf_predict(clf, x_val)
y_proba_test = clf_predict(clf, x_test)

In [ ]:
target_type = "csr_matrix"

if target_type == "torch":
    # Works also for torch tensors
    import torch 
    y_test = torch.tensor(y_test)
    y_val = torch.tensor(y_val)
    y_proba_test = torch.tensor(y_proba_test)
    y_proba_val = torch.tensor(y_proba_val)
    
elif target_type == "csr_matrix":
    # Works also for csr_sparse matrices
    from scipy.sparse import csr_matrix
    y_test = csr_matrix(y_test)
    y_val = csr_matrix(y_val)
    y_proba_test = csr_matrix(y_proba_test)
    y_proba_val = csr_matrix(y_proba_val)

In [ ]:
from xcolumns.confusion_matrix import calculate_confusion_matrix
from xcolumns.metrics import macro_fmeasure_on_conf_matrix

def evaluate(y_true, y_pred):
    """
    Evaluate the performance of a multioutput classifier.
    """
    C = calculate_confusion_matrix(y_true, y_pred)
    print(macro_fmeasure_on_conf_matrix(C.tp, C.fp, C.fn, C.tn))


In [ ]:
# Test top-k prediction
from xcolumns.weighted_prediction import predict_top_k

y_pred = predict_top_k(y_proba_test, k=3)
evaluate(y_test, y_pred)

In [ ]:
# Frank Wolfe
from xcolumns.frank_wolfe import find_optimal_randomized_classifier_using_frank_wolfe

rnd_clf, meta = find_optimal_randomized_classifier_using_frank_wolfe(
    y_val, y_proba_val, macro_fmeasure_on_conf_matrix, k = 3, grad_func="torch", return_meta=True, seed=seed
)

y_pred = rnd_clf.predict(y_proba_test, seed=seed)
print(meta["time"])
evaluate(y_test, y_pred)